# Nemotron Next 8B LoRA Fine-tuning with FiQA Dataset

This notebook demonstrates LoRA fine-tuning of [NVIDIA Nemotron Next 8B](https://huggingface.co/nvidia/Nemotron-Next-8B) on the [FiQA dataset](https://huggingface.co/datasets/explodinggradients/fiqa) for financial question answering.

## Table of Contents

1. [Setup & Environment](#setup)
2. [Model Loading](#model-loading)
3. [Dataset Loading & Preprocessing](#dataset)
4. [Baseline Evaluation](#baseline)
5. [LoRA Configuration](#lora-config)
6. [LoRA Training](#training)
7. [Fine-tuned Evaluation](#evaluation)
8. [Visualization & Analysis](#visualization)

---

## GPU Requirements

⚠️ **This notebook requires a GPU with 24GB+ VRAM** (A100, H100, or RTX 4090 recommended)

| Phase | GPU Required | Time Estimate |
|-------|--------------|---------------|
| Model Loading | ✅ Yes | 2-5 min |
| Dataset Prep | ✅ Yes | 10-25 min |
| Baseline Eval | ✅ Yes | 30-60 min |
| LoRA Training | ✅ Yes | 3-6 hours |
| Final Eval | ✅ Yes | 30-60 min |


<a name="setup"></a>
## 1. Setup & Environment

First, let's verify our environment and import required libraries.


In [ ]:
# GPU REQUIRED - Verify CUDA availability
import torch

print("=" * 50)
print("Environment Check")
print("=" * 50)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    raise RuntimeError("❌ CUDA not available! This notebook requires a GPU.")

print("=" * 50)
print("✅ GPU environment verified!")


In [ ]:
# Core imports
import os
import json
import time
from pathlib import Path
from typing import Dict, List, Optional, Tuple

# Data processing
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from tqdm.auto import tqdm

# NeMo AutoModel imports
try:
    import nemo_automodel
    from nemo_automodel._transformers import NeMoAutoModelForCausalLM
    from nemo_automodel.components._peft.lora import PeftConfig, apply_lora_to_linear_modules
    print(f"✅ NeMo AutoModel imported successfully")
except ImportError as e:
    print(f"⚠️ NeMo AutoModel not found: {e}")
    print("Please install: cd Automodel && uv pip install -e .")
    raise

# Transformers for tokenizer
from transformers import AutoTokenizer, AutoProcessor

# Evaluation
import evaluate

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✅ All imports successful!")


In [ ]:
# Configuration
CONFIG = {
    # Model
    "model_name": "nvidia/Nemotron-Next-8B",
    "torch_dtype": torch.bfloat16,
    
    # Dataset
    "dataset_name": "explodinggradients/fiqa",
    "train_split_ratio": 0.8,  # 80% train, 20% validation from original train
    "max_length": 512,
    
    # LoRA
    "lora_rank": 8,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    
    # Training
    "learning_rate": 2e-4,
    "batch_size": 4,
    "num_epochs": 3,
    "gradient_accumulation_steps": 4,
    "warmup_ratio": 0.1,
    
    # Paths
    "output_dir": "./outputs",
    "checkpoint_dir": "./checkpoints",
}

# Create output directories
Path(CONFIG["output_dir"]).mkdir(parents=True, exist_ok=True)
Path(CONFIG["checkpoint_dir"]).mkdir(parents=True, exist_ok=True)

print("✅ Configuration loaded")
print(f"   Model: {CONFIG['model_name']}")
print(f"   Dataset: {CONFIG['dataset_name']}")
print(f"   LoRA rank: {CONFIG['lora_rank']}, alpha: {CONFIG['lora_alpha']}")


---

<a name="model-loading"></a>
## 2. Model Loading

**⏱️ Time Estimate: 2-5 minutes** | **GPU REQUIRED**

Load Nemotron Next 8B using NeMo AutoModel APIs.


In [ ]:
# GPU REQUIRED - Load Nemotron Next 8B
# ⏱️ Time: 2-5 minutes

print("=" * 50)
print("Loading Nemotron Next 8B...")
print("=" * 50)

start_time = time.time()

# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    CONFIG["model_name"],
    trust_remote_code=True,
)

# Ensure padding token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"✅ Tokenizer loaded")
print(f"   Vocab size: {tokenizer.vocab_size}")
print(f"   Pad token: {tokenizer.pad_token}")

# Load model using NeMo AutoModel
print("\nLoading model (this may take a few minutes)...")
model = NeMoAutoModelForCausalLM.from_pretrained(
    CONFIG["model_name"],
    torch_dtype=CONFIG["torch_dtype"],
    trust_remote_code=True,
    device_map="auto",
)

load_time = time.time() - start_time

print(f"\n✅ Model loaded successfully in {load_time:.1f} seconds")
print(f"   Model type: {type(model).__name__}")
print(f"   Dtype: {model.dtype}")
print(f"   Device: {next(model.parameters()).device}")


In [ ]:
# Print model architecture summary
def count_parameters(model):
    """Count trainable and total parameters."""
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

total, trainable = count_parameters(model)

print("=" * 50)
print("Model Summary")
print("=" * 50)
print(f"Total parameters: {total:,} ({total/1e9:.2f}B)")
print(f"Trainable parameters: {trainable:,} ({trainable/1e9:.2f}B)")
print(f"Trainable %: {100 * trainable / total:.2f}%")


In [ ]:
# Test basic inference to verify model works
print("=" * 50)
print("Testing Model Inference")
print("=" * 50)

# Simple financial question to test
test_prompt = """You are a helpful financial assistant.

Question: What is the difference between a stock and a bond?

Answer:"""

# Tokenize
inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)

# Generate
print("Generating response...")
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
    )

# Decode
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\n" + "=" * 50)
print("Test Prompt:")
print("=" * 50)
print(test_prompt)
print("\n" + "=" * 50)
print("Model Response:")
print("=" * 50)
print(response[len(test_prompt):].strip())
print("\n✅ Model inference test passed!")


---

<a name="dataset"></a>
## 3. Dataset Loading & Preprocessing

**⏱️ Time Estimate: 10-25 minutes** | **GPU REQUIRED**

Load FiQA dataset and format for instruction fine-tuning.

### FiQA Dataset Info
- **Source**: [explodinggradients/fiqa](https://huggingface.co/datasets/explodinggradients/fiqa)
- **Splits**: Train (5,500), Validation (500), Test (648)
- **Format**: Question + Ground Truth Answers


In [ ]:
# Load FiQA dataset
# ⏱️ Time: ~1 minute

print("=" * 50)
print("Loading FiQA Dataset")
print("=" * 50)

# Load dataset with 'main' config
fiqa_dataset = load_dataset("explodinggradients/fiqa", "main")

print(f"✅ Dataset loaded!")
print(f"\nSplits:")
for split_name, split_data in fiqa_dataset.items():
    print(f"  {split_name}: {len(split_data):,} samples")

print(f"\nColumns: {fiqa_dataset['train'].column_names}")


In [ ]:
# Explore sample data
print("=" * 50)
print("Sample Data")
print("=" * 50)

sample = fiqa_dataset['train'][0]
print(f"\nQuestion:\n{sample['question']}")
print(f"\nGround Truth Answer (first of {len(sample['ground_truths'])}):")
print(sample['ground_truths'][0][:500] + "..." if len(sample['ground_truths'][0]) > 500 else sample['ground_truths'][0])


In [ ]:
# Format data for instruction fine-tuning
def format_for_training(example: Dict) -> Dict:
    """
    Format FiQA sample for instruction fine-tuning.
    
    Input format:
        - question: str
        - ground_truths: List[str]
    
    Output format:
        - text: formatted instruction string
        - input_text: question only (for inference)
        - target_text: answer only (for evaluation)
    """
    question = example['question']
    # Use first ground truth answer (most relevant)
    answer = example['ground_truths'][0] if example['ground_truths'] else ""
    
    # Truncate very long answers to avoid context length issues
    max_answer_len = 1024
    if len(answer) > max_answer_len:
        answer = answer[:max_answer_len] + "..."
    
    # Format as instruction-following
    instruction = f"""You are a helpful financial assistant. Answer the following question about finance.

Question: {question}

Answer: {answer}"""
    
    # Also store components separately for evaluation
    input_prompt = f"""You are a helpful financial assistant. Answer the following question about finance.

Question: {question}

Answer:"""
    
    return {
        "text": instruction,
        "input_text": input_prompt,
        "target_text": answer,
        "question": question,
    }

print("✅ Formatting function defined")


In [ ]:
# Apply formatting to all splits
# ⏱️ Time: ~1 minute

print("=" * 50)
print("Formatting Dataset")
print("=" * 50)

formatted_dataset = fiqa_dataset.map(
    format_for_training,
    remove_columns=['ground_truths'],  # Keep only new columns
    desc="Formatting samples"
)

print(f"\n✅ Dataset formatted!")
print(f"New columns: {formatted_dataset['train'].column_names}")

# Show formatted example
print("\n" + "=" * 50)
print("Formatted Sample")
print("=" * 50)
print(formatted_dataset['train'][0]['text'][:800] + "...")


In [ ]:
# Tokenize dataset
# ⏱️ Time: 5-15 minutes

def tokenize_function(examples: Dict) -> Dict:
    """Tokenize examples for training."""
    # Tokenize the full text (question + answer)
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        max_length=CONFIG['max_length'],
        padding='max_length',
        return_tensors=None,
    )
    
    # For causal LM, labels are same as input_ids
    tokenized['labels'] = tokenized['input_ids'].copy()
    
    return tokenized

print("=" * 50)
print("Tokenizing Dataset")
print("=" * 50)

tokenized_dataset = formatted_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=['text', 'input_text', 'target_text', 'question'],
    desc="Tokenizing"
)

print(f"\n✅ Dataset tokenized!")
print(f"Columns: {tokenized_dataset['train'].column_names}")
print(f"Sample input_ids length: {len(tokenized_dataset['train'][0]['input_ids'])}")


In [ ]:
# Dataset summary
print("=" * 50)
print("Dataset Preparation Complete")
print("=" * 50)

print("\n📊 Dataset Statistics:")
print(f"   Train samples: {len(tokenized_dataset['train']):,}")
print(f"   Validation samples: {len(tokenized_dataset['validation']):,}")
print(f"   Test samples: {len(tokenized_dataset['test']):,}")
print(f"   Max sequence length: {CONFIG['max_length']}")

# Keep the formatted dataset for evaluation later
eval_dataset = formatted_dataset  # Has input_text and target_text

print("\n✅ Phase 3: Dataset Loading & Preprocessing complete!")


---

<a name="baseline"></a>
## 4. Baseline Evaluation

**⏱️ Time Estimate: 30-60 minutes** | **GPU REQUIRED**

Evaluate the base model on FiQA test set before fine-tuning.


In [ ]:
# Baseline Evaluation Setup
# ⏱️ Time: ~30-60 minutes depending on test set size

# Use a subset for baseline to save time
BASELINE_SAMPLE_SIZE = 100  # Evaluate on 100 samples for speed

# Select random samples from test set
import random
random.seed(42)

test_indices = random.sample(range(len(eval_dataset['test'])), min(BASELINE_SAMPLE_SIZE, len(eval_dataset['test'])))
baseline_test_samples = eval_dataset['test'].select(test_indices)

print(f"📊 Baseline evaluation on {len(baseline_test_samples)} test samples")
print(f"   (Full test set has {len(eval_dataset['test'])} samples)")


In [ ]:
# Define inference function
def generate_response(prompt: str, max_new_tokens: int = 256) -> str:
    """Generate response from model given a prompt."""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=CONFIG['max_length'])
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # Decode only the generated part (exclude input)
    input_length = inputs['input_ids'].shape[1]
    generated_tokens = outputs[0][input_length:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    return response.strip()

print("✅ Inference function defined")


In [ ]:
# Define evaluation metrics
from collections import Counter
import re

def normalize_text(text: str) -> str:
    """Normalize text for comparison."""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = ' '.join(text.split())
    return text

def compute_exact_match(prediction: str, reference: str) -> float:
    """Compute exact match score (0 or 1)."""
    return float(normalize_text(prediction) == normalize_text(reference))

def compute_f1(prediction: str, reference: str) -> float:
    """Compute token-level F1 score."""
    pred_tokens = normalize_text(prediction).split()
    ref_tokens = normalize_text(reference).split()
    
    if len(pred_tokens) == 0 or len(ref_tokens) == 0:
        return float(pred_tokens == ref_tokens)
    
    common = Counter(pred_tokens) & Counter(ref_tokens)
    num_common = sum(common.values())
    
    if num_common == 0:
        return 0.0
    
    precision = num_common / len(pred_tokens)
    recall = num_common / len(ref_tokens)
    f1 = 2 * precision * recall / (precision + recall)
    
    return f1

def compute_bleu_simple(prediction: str, reference: str, max_n: int = 4) -> float:
    """Compute simple BLEU score (without brevity penalty for simplicity)."""
    pred_tokens = normalize_text(prediction).split()
    ref_tokens = normalize_text(reference).split()
    
    if len(pred_tokens) == 0:
        return 0.0
    
    scores = []
    for n in range(1, min(max_n + 1, len(pred_tokens) + 1)):
        pred_ngrams = Counter([tuple(pred_tokens[i:i+n]) for i in range(len(pred_tokens) - n + 1)])
        ref_ngrams = Counter([tuple(ref_tokens[i:i+n]) for i in range(len(ref_tokens) - n + 1)])
        
        matches = sum((pred_ngrams & ref_ngrams).values())
        total = sum(pred_ngrams.values())
        
        if total > 0:
            scores.append(matches / total)
        else:
            scores.append(0.0)
    
    if not scores or all(s == 0 for s in scores):
        return 0.0
    
    # Geometric mean
    from functools import reduce
    import operator
    non_zero = [s for s in scores if s > 0]
    if not non_zero:
        return 0.0
    return reduce(operator.mul, non_zero, 1) ** (1 / len(non_zero))

print("✅ Evaluation metrics defined")


In [ ]:
# Run baseline evaluation
# ⏱️ Time: 30-60 minutes (depends on GPU and sample size)

print("=" * 50)
print("Running Baseline Evaluation (Base Model)")
print("=" * 50)
print(f"Samples: {len(baseline_test_samples)}")
print("This may take a while...")
print()

baseline_results = []

for idx, sample in enumerate(tqdm(baseline_test_samples, desc="Evaluating")):
    input_prompt = sample['input_text']
    reference = sample['target_text']
    
    # Generate response
    prediction = generate_response(input_prompt)
    
    # Compute metrics
    em = compute_exact_match(prediction, reference)
    f1 = compute_f1(prediction, reference)
    bleu = compute_bleu_simple(prediction, reference)
    
    baseline_results.append({
        'question': sample['question'],
        'reference': reference,
        'prediction': prediction,
        'exact_match': em,
        'f1': f1,
        'bleu': bleu,
    })
    
    # Show progress every 10 samples
    if (idx + 1) % 10 == 0:
        avg_f1 = sum(r['f1'] for r in baseline_results) / len(baseline_results)
        print(f"  Progress: {idx + 1}/{len(baseline_test_samples)} | Running F1: {avg_f1:.4f}")

print("\n✅ Baseline evaluation complete!")


In [ ]:
# Compute and display baseline metrics
baseline_df = pd.DataFrame(baseline_results)

baseline_metrics = {
    'exact_match': baseline_df['exact_match'].mean(),
    'f1': baseline_df['f1'].mean(),
    'bleu': baseline_df['bleu'].mean(),
}

print("=" * 50)
print("📊 BASELINE METRICS (Pre-Fine-tuning)")
print("=" * 50)
print(f"\n  Exact Match:  {baseline_metrics['exact_match']:.4f}")
print(f"  F1 Score:     {baseline_metrics['f1']:.4f}")
print(f"  BLEU Score:   {baseline_metrics['bleu']:.4f}")
print()

# Show a few example predictions
print("=" * 50)
print("Sample Predictions")
print("=" * 50)
for i in range(min(3, len(baseline_results))):
    result = baseline_results[i]
    print(f"\n📌 Sample {i+1}:")
    print(f"   Question: {result['question'][:100]}...")
    print(f"   Reference: {result['reference'][:150]}...")
    print(f"   Prediction: {result['prediction'][:150]}...")
    print(f"   F1: {result['f1']:.4f}")


In [ ]:
# Save baseline results for later comparison
import json
import os

# Create outputs directory
os.makedirs("outputs", exist_ok=True)

# Save baseline metrics
baseline_save_path = "outputs/baseline_metrics.json"
with open(baseline_save_path, 'w') as f:
    json.dump({
        'metrics': baseline_metrics,
        'sample_size': len(baseline_results),
        'model_name': CONFIG['model_name'],
    }, f, indent=2)

# Save detailed results
baseline_results_path = "outputs/baseline_results.csv"
baseline_df.to_csv(baseline_results_path, index=False)

print(f"✅ Baseline metrics saved to: {baseline_save_path}")
print(f"✅ Detailed results saved to: {baseline_results_path}")
print(f"\n✅ Phase 4: Baseline Evaluation complete!")


---

<a name="lora-config"></a>
## 5. LoRA Configuration

**⏱️ Time Estimate: ~5 minutes** | **GPU REQUIRED**

Configure and apply LoRA adapter to the model using PEFT (Parameter-Efficient Fine-Tuning).


In [ ]:
# Import PEFT for LoRA
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

print("=" * 50)
print("LoRA Configuration")
print("=" * 50)

# Define LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=CONFIG['lora_r'],                    # Rank of the update matrices
    lora_alpha=CONFIG['lora_alpha'],       # Scaling factor
    lora_dropout=CONFIG['lora_dropout'],   # Dropout probability
    target_modules=[                        # Which modules to apply LoRA to
        "q_proj",
        "k_proj", 
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    bias="none",                           # Don't train bias terms
)

print(f"✅ LoRA Config:")
print(f"   Rank (r): {lora_config.r}")
print(f"   Alpha: {lora_config.lora_alpha}")
print(f"   Dropout: {lora_config.lora_dropout}")
print(f"   Target modules: {lora_config.target_modules}")


In [ ]:
# Apply LoRA adapter to the model
print("=" * 50)
print("Applying LoRA Adapter")
print("=" * 50)

# Enable gradient checkpointing for memory efficiency
model.gradient_checkpointing_enable()

# Apply LoRA
model = get_peft_model(model, lora_config)

print("✅ LoRA adapter applied to model")


In [ ]:
# Verify trainable parameters
print("=" * 50)
print("Trainable Parameters")
print("=" * 50)

# Print trainable parameters summary
model.print_trainable_parameters()

# Detailed breakdown
trainable_params = 0
all_params = 0
for name, param in model.named_parameters():
    all_params += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()

print(f"\n📊 Parameter Summary:")
print(f"   Total parameters:     {all_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Trainable %:          {100 * trainable_params / all_params:.4f}%")
print(f"   Memory reduction:     ~{100 - 100 * trainable_params / all_params:.1f}%")

print(f"\n✅ Phase 5: LoRA Configuration complete!")


---

<a name="training"></a>
## 6. LoRA Training

**⏱️ Time Estimate: 3-6 hours** | **GPU REQUIRED**

Train the LoRA adapter on FiQA training data.

> 🚧 **TODO: Phase 6** - Implement LoRA training


---

<a name="evaluation"></a>
## 7. Fine-tuned Evaluation

**⏱️ Time Estimate: 30-60 minutes** | **GPU REQUIRED**

Evaluate the fine-tuned model and compare with baseline.

> 🚧 **TODO: Phase 7** - Implement fine-tuned evaluation


---

<a name="visualization"></a>
## 8. Visualization & Analysis

Create visualizations comparing baseline vs fine-tuned performance.

> 🚧 **TODO: Phase 8** - Implement visualization and analysis


---

## Summary

This notebook demonstrated LoRA fine-tuning of Nemotron Next 8B on the FiQA financial QA dataset.

### Results

| Metric | Baseline | Fine-tuned | Improvement |
|--------|----------|------------|-------------|
| Exact Match | TBD | TBD | TBD |
| F1 Score | TBD | TBD | TBD |
| BLEU | TBD | TBD | TBD |

### Next Steps

- Experiment with different LoRA ranks
- Try longer training
- Evaluate on additional financial QA datasets
